## Generate diagnostic plots

Show gas/stellar surface densities, velocities, and velocity dispersions. 

In [ ]:
import os
from glob import glob
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from time import process_time 
from copy import copy
import multiprocessing as mp


In [ ]:
data_dir = '/home/bottrell/scratch/Fire_Kinematics/LOSVD'
sim_tag = 'G2G3_e'
sub_tag = 'orbit_1'
camera = 'i0'
ptype = 'gas'
snapID = 270

losvd_name = '{}/{}/{}/kinemetry_datacube_{}-{}_{:04d}_{}_{}.fits'.format(data_dir,sim_tag,sub_tag,sim_tag,sub_tag,snapID,ptype,camera)
if os.access(losvd_name,0): print('File found.')

losvd_head = fits.getheader(losvd_name)
losvd_data = fits.getdata(losvd_name)

### Create single diagnostic plot

In [ ]:
def add_colorbar(im,ax,label,colour='white'):
    axin = ax.inset_axes(
            [0.05,0.05, 0.9, 0.025], transform=ax.transAxes)
    cbar = plt.colorbar(im, cax=axin, orientation='horizontal')
    cbar.ax.tick_params(color=colour, labelcolor=colour,direction='in',labeltop=1,
                        labelbottom=0,top=1,bottom=0, width=1, labelsize=14)
    cbar.set_label(label, color=colour, size=12)
    cbar.ax.xaxis.set_label_position('top') 
    cbar.outline.set_edgecolor(colour)
    cbar.outline.set_linewidth(1.5)

def generate_maps(losvd_name):
    if not os.access(losvd_name,0):
        print('LOSVD file not found')
    else:
        losvd_head = fits.getheader(losvd_name)
        losvd_data = fits.getdata(losvd_name)
        losvd_shape = losvd_data.shape
        vlim = losvd_head['VLIM'] # km/s
        delv = losvd_head['DELV'] # km/s
        fov = losvd_head['FOVSIZE']/1000. # kpc
        vel = np.linspace(-vlim,vlim,losvd_shape[-1],endpoint=False)+delv/2.
        sum_wi = np.nansum(losvd_data,axis=-1)
        sum_wivi = np.nansum(losvd_data*vel,axis=-1)
        vbar = sum_wivi/sum_wi
        Nprime = np.nansum(losvd_data>0,axis=-1)
        vstd = np.nansum(losvd_data*(vel-vbar[...,np.newaxis])**2,axis=-1)
        vstd /= (Nprime-1)/Nprime*sum_wi
        vstd = np.sqrt(vstd)
        losvd_maps = np.array([sum_wi,vbar,vstd])
        return losvd_maps

def generate_figure(sim_tag,sub_tag,camera,snapID):
    
    data_dir = '/home/bottrell/scratch/Fire_Kinematics/LOSVD'
    
    fig,axarr = plt.subplots(2,3,figsize=(15,10))
    fig.subplots_adjust(wspace=0.05,hspace=0.05)
    axarr = axarr.flat

    for ax in axarr:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        for spine in ['top','left','right','bottom']:
            ax.spines[spine].set_linewidth(4)

    ptype = 'gas'
    losvd_name = '{}/{}/{}/kinemetry_datacube_{}-{}_{:04d}_{}_{}.fits'.format(data_dir,sim_tag,sub_tag,sim_tag,sub_tag,snapID,ptype,camera)
    maps = generate_maps(losvd_name)

    ax = axarr[0]
    cmap = copy(plt.cm.inferno)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(np.log10(maps[0]),vmin=3,vmax=7,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Gas surface density, $\log\;\Sigma_{\mathrm{gas}}$ [M$_{\odot}}$/kpc$^2$]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ax = axarr[1]
    cmap = copy(plt.cm.jet)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[1],vmin=-150,vmax=150,cmap=cmap,origin='lower',aspect='auto',interpolation='none')
    cbar_colour = 'white'
    label = r'Gas velocity, $V_{\mathrm{LOS,gas}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ax = axarr[2]
    cmap = copy(plt.cm.Blues_r)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[2],vmin=0,vmax=150,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Gas vel. dispersion, $\sigma_{\mathrm{LOS,gas}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ptype = 'stars'
    losvd_name = '{}/{}/{}/kinemetry_datacube_{}-{}_{:04d}_{}_{}.fits'.format(data_dir,sim_tag,sub_tag,sim_tag,sub_tag,snapID,ptype,camera)
    maps = generate_maps(losvd_name)
    mask = np.log10(maps[0])<4

    ax = axarr[3]
    cmap = copy(plt.cm.bone)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(np.log10(maps[0]),vmin=3,vmax=7,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Stellar surface density, $\log\;\Sigma_{\star}$ [M$_{\odot}}$/kpc$^2$]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    maps[:,mask] = np.nan

    ax = axarr[4]
    cmap = copy(plt.cm.jet)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[1],vmin=-100,vmax=100,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Stellar velocity, $V_{\mathrm{LOS,\star}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ax = axarr[5]
    cmap = copy(plt.cm.Blues_r)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[2],vmin=0,vmax=150,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Stellar vel. dispersion, $\sigma_{\mathrm{LOS,\star}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)
    
    return fig

sim_tag = 'G2G3_e'
sub_tag = 'orbit_1'
camera = 'i0'
snapID = 279

fig = generate_figure(sim_tag,sub_tag,camera,snapID)

save_dir = '/home/bottrell/scratch/Fire_Kinematics/Figures/Movies'
fig.savefig('{}/Diagnostic_{}_{}_{:04d}_{}_LowRes.png'.format(save_dir,sim_tag,sub_tag,snapID,camera),
            bbox_inches='tight',dpi=128)

### Create all diagnostic plots for given merger w/ multiprocessing

In [ ]:
import os,imageio
from glob import glob
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from time import process_time 
from copy import copy
import multiprocessing as mp

def add_colorbar(im,ax,label,colour='white'):
    axin = ax.inset_axes(
            [0.05,0.05, 0.9, 0.025], transform=ax.transAxes)
    cbar = plt.colorbar(im, cax=axin, orientation='horizontal')
    cbar.ax.tick_params(color=colour, labelcolor=colour,direction='in',labeltop=1,
                        labelbottom=0,top=1,bottom=0, width=1, labelsize=14)
    cbar.set_label(label, color=colour, size=12)
    cbar.ax.xaxis.set_label_position('top') 
    cbar.outline.set_edgecolor(colour)
    cbar.outline.set_linewidth(1.5)

def generate_maps(losvd_name):
    if not os.access(losvd_name,0):
        print('LOSVD file not found')
    else:
        losvd_head = fits.getheader(losvd_name)
        losvd_data = fits.getdata(losvd_name)
        losvd_shape = losvd_data.shape
        vlim = losvd_head['VLIM'] # km/s
        delv = losvd_head['DELV'] # km/s
        fov = losvd_head['FOVSIZE']/1000. # kpc
        vel = np.linspace(-vlim,vlim,losvd_shape[-1],endpoint=False)+delv/2.
        sum_wi = np.nansum(losvd_data,axis=-1)
        sum_wivi = np.nansum(losvd_data*vel,axis=-1)
        vbar = sum_wivi/sum_wi
        Nprime = np.nansum(losvd_data>0,axis=-1)
        vstd = np.nansum(losvd_data*(vel-vbar[...,np.newaxis])**2,axis=-1)
        vstd /= (Nprime-1)/Nprime*sum_wi
        vstd = np.sqrt(vstd)
        losvd_maps = np.array([sum_wi,vbar,vstd])
        return losvd_maps

def generate_figure(sim_tag,sub_tag,camera,snapID):
    
    data_dir = '/home/bottrell/scratch/Fire_Kinematics/LOSVD'
    
    fig,axarr = plt.subplots(2,3,figsize=(15,10))
    fig.subplots_adjust(wspace=0.05,hspace=0.05)
    axarr = axarr.flat

    for ax in axarr:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        for spine in ['top','left','right','bottom']:
            ax.spines[spine].set_linewidth(4)

    ptype = 'gas'
    losvd_name = '{}/{}/{}/kinemetry_datacube_{}-{}_{:04d}_{}_{}.fits'.format(data_dir,sim_tag,sub_tag,sim_tag,sub_tag,snapID,ptype,camera)
    maps = generate_maps(losvd_name)

    ax = axarr[0]
    cmap = copy(plt.cm.inferno)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(np.log10(maps[0]),vmin=3,vmax=7,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Gas surface density, $\log\;\Sigma_{\mathrm{gas}}$ [M$_{\odot}}$/kpc$^2$]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ax = axarr[1]
    cmap = copy(plt.cm.jet)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[1],vmin=-150,vmax=150,cmap=cmap,origin='lower',aspect='auto',interpolation='none')
    cbar_colour = 'white'
    label = r'Gas velocity, $V_{\mathrm{LOS,gas}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ax = axarr[2]
    cmap = copy(plt.cm.Blues_r)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[2],vmin=0,vmax=150,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Gas vel. dispersion, $\sigma_{\mathrm{LOS,gas}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ptype = 'stars'
    losvd_name = '{}/{}/{}/kinemetry_datacube_{}-{}_{:04d}_{}_{}.fits'.format(data_dir,sim_tag,sub_tag,sim_tag,sub_tag,snapID,ptype,camera)
    maps = generate_maps(losvd_name)
    mask = np.log10(maps[0])<4

    ax = axarr[3]
    cmap = copy(plt.cm.bone)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(np.log10(maps[0]),vmin=3,vmax=7,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Stellar surface density, $\log\;\Sigma_{\star}$ [M$_{\odot}}$/kpc$^2$]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    maps[:,mask] = np.nan

    ax = axarr[4]
    cmap = copy(plt.cm.jet)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[1],vmin=-100,vmax=100,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Stellar velocity, $V_{\mathrm{LOS,\star}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)

    ax = axarr[5]
    cmap = copy(plt.cm.Blues_r)
    cmap.set_bad('black', 1.0)
    im = ax.imshow(maps[2],vmin=0,vmax=150,cmap=cmap,origin='lower',aspect='auto',interpolation=None)
    cbar_colour = 'white'
    label = r'Stellar vel. dispersion, $\sigma_{\mathrm{LOS,\star}}$ [km/s]'
    add_colorbar(im,ax,label,colour=cbar_colour)
    
    return fig

def worker_wrapper(args):
    sim_tag,sub_tag,camera,snapID = args
    fig = generate_figure(sim_tag,sub_tag,camera,snapID)
    fig_name = '{}/Diagnostic_{}_{}_{:04d}_{}.png'.format(save_dir,sim_tag,sub_tag,snapID,camera)
    if not os.access(fig_name,0): 
        fig.savefig(fig_name,bbox_inches='tight',dpi=512)
    
losvd_dir = '/home/bottrell/scratch/Fire_Kinematics/LOSVD'
galsep_dir = '/home/bottrell/scratch/Fire/GalSep'
save_dir = '/home/bottrell/scratch/Fire_Kinematics/Figures/Movies'
sim_tag = 'G2G3_e'
sub_tag = 'orbit_1'
camera = 'i0'

galsep_pair = '{}/galsep_{}_{}_PairSnapIDs.npz'.format(galsep_dir,sim_tag,sub_tag)
galsep_post = '{}/galsep_{}_{}_PMSnapIDs.npz'.format(galsep_dir,sim_tag,sub_tag)
snapIDs_pair = np.load(galsep_pair)['snapID'].astype(int)
snapIDs_post = np.load(galsep_post)['snapID'].astype(int)
snapIDs = np.concatenate((snapIDs_pair,snapIDs_post))

argList = []
for snapID in snapIDs:
    argList += [(sim_tag,sub_tag,camera,snapID),]
    
SLURM_CPUS = mp.cpu_count()
pool = mp.Pool(SLURM_CPUS)
pool.map(worker_wrapper, argList)
pool.close()
pool.join()


### Combine diagnostic plots into movie

In [ ]:
import os,imageio
from glob import glob
import matplotlib
matplotlib.use('Agg')

losvd_dir = '/home/bottrell/scratch/Fire_Kinematics/LOSVD'
galsep_dir = '/home/bottrell/scratch/Fire/GalSep'
save_dir = '/home/bottrell/scratch/Fire_Kinematics/Figures/Movies'
sim_tag = 'G2G3_e'
sub_tag = 'orbit_1'
camera = 'i0'

file_names = list(sorted(glob('{}/Diagnostic_{}_{}_????_{}.png'.format(save_dir,sim_tag,sub_tag,camera))))
movie_name = '{}/Diagnostic_{}_{}_{}.mp4'.format(save_dir,sim_tag,sub_tag,camera)

with imageio.get_writer(movie_name) as writer:
    for file_name in file_names:
        image = imageio.imread(file_name)
        writer.append_data(image)